In [1]:
import torch
import torch.nn as nn
from model import GlobalPredictor

In [2]:
import os

In [3]:
import pickle as pk

In [4]:
import numpy as np

In [5]:
global_predictor = torch.load('./results_osaka/global_predictor_fl.pytorch').cuda(0)

In [6]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [7]:
T = 96
dT = 4
batch_size = 256

In [8]:
mrr_avg_rank_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInOsakaProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)
    
    mrr = 0.0
    avg_rank = 0.0
    acc5 = 0.0
    
    with open(filename, 'rb') as f:
        data = pk.load(f)
        data = torch.LongTensor(list(data.values())).cuda(0)
        with torch.no_grad():
            for t in range(T - 2 * dT):
                for i in range(0, data.shape[0], batch_size):
                    data_x = data[i: i + batch_size, t: t + dT]
                    data_t = torch.zeros_like(data_x) + t
                    data_y = data[i: i + batch_size, t + 2 * dT - 1].cpu().numpy()
                    pred = global_predictor.predict(data_x, data_t).cpu().numpy()
                    rank = np.where((np.argsort(-pred, axis=1, ).T - data_y).T == 0)[1] + 1
                    mrr += np.sum(1 / rank)
                    avg_rank += np.sum(rank)
                    acc5 += np.sum(rank <= 5)
            
            sample_size = data.shape[0] * (T - 2 * dT)
            print('Evaluation: mrr = {:.4f}, avg_rank = {:.4f}, acc@5 = {:.4f}'.format(mrr / sample_size, avg_rank / sample_size, acc5 / sample_size))
            mrr_avg_rank_dict[(y, m, d)] = (mrr / sample_size, avg_rank / sample_size, acc5 / sample_size)

/data/fan/UsersInOsakaProcessed/20121201_interp.pk
Evaluation: mrr = 0.7975, avg_rank = 24.9238, acc@5 = 0.8401
/data/fan/UsersInOsakaProcessed/20121202_interp.pk
Evaluation: mrr = 0.8165, avg_rank = 22.1845, acc@5 = 0.8563
/data/fan/UsersInOsakaProcessed/20121203_interp.pk
Evaluation: mrr = 0.7920, avg_rank = 25.1171, acc@5 = 0.8355
/data/fan/UsersInOsakaProcessed/20121204_interp.pk
Evaluation: mrr = 0.7907, avg_rank = 25.5204, acc@5 = 0.8340
/data/fan/UsersInOsakaProcessed/20121205_interp.pk
Evaluation: mrr = 0.7902, avg_rank = 25.7969, acc@5 = 0.8337
/data/fan/UsersInOsakaProcessed/20121206_interp.pk
Evaluation: mrr = 0.7899, avg_rank = 25.7988, acc@5 = 0.8335
/data/fan/UsersInOsakaProcessed/20121207_interp.pk
Evaluation: mrr = 0.7856, avg_rank = 25.9758, acc@5 = 0.8301
/data/fan/UsersInOsakaProcessed/20121208_interp.pk
Evaluation: mrr = 0.7994, avg_rank = 24.5075, acc@5 = 0.8419
/data/fan/UsersInOsakaProcessed/20121209_interp.pk
Evaluation: mrr = 0.8162, avg_rank = 21.8651, acc@5 =

Evaluation: mrr = 0.7932, avg_rank = 25.9432, acc@5 = 0.8350
/data/fan/UsersInOsakaProcessed/20130213_interp.pk
Evaluation: mrr = 0.7912, avg_rank = 26.1572, acc@5 = 0.8340
/data/fan/UsersInOsakaProcessed/20130214_interp.pk
Evaluation: mrr = 0.7903, avg_rank = 25.9678, acc@5 = 0.8339
/data/fan/UsersInOsakaProcessed/20130215_interp.pk
Evaluation: mrr = 0.7884, avg_rank = 25.8036, acc@5 = 0.8316
/data/fan/UsersInOsakaProcessed/20130216_interp.pk
Evaluation: mrr = 0.8031, avg_rank = 24.8943, acc@5 = 0.8444
/data/fan/UsersInOsakaProcessed/20130217_interp.pk
Evaluation: mrr = 0.8230, avg_rank = 21.9713, acc@5 = 0.8614
/data/fan/UsersInOsakaProcessed/20130218_interp.pk
Evaluation: mrr = 0.7976, avg_rank = 24.8889, acc@5 = 0.8386
/data/fan/UsersInOsakaProcessed/20130219_interp.pk
Evaluation: mrr = 0.7928, avg_rank = 25.7679, acc@5 = 0.8351
/data/fan/UsersInOsakaProcessed/20130220_interp.pk
Evaluation: mrr = 0.7906, avg_rank = 26.2173, acc@5 = 0.8340
/data/fan/UsersInOsakaProcessed/20130221_in

In [9]:
with open('./results_osaka/evaluation_global_fl_rank.pk', 'wb') as f:
    pk.dump(mrr_avg_rank_dict, f)